In [ ]:
import geopandas as gpd

In [ ]:
amenities_file = '/app/data/amenities'

amn = gpd.read_file(amenities_file)
amn['name'].fillna('', inplace=True)

idna = amn['id'].isna()
amn.loc[idna, 'id'] = ('2024' + amn[idna].index.astype(str)).astype(int)

futr = amn['name'].str.contains('futur', case=False)
amn.loc[ futr, 'scenario'] = 'actual'
amn.loc[~futr, 'scenario'] = 'future'

amn_cols = ['id', 'Category', 'scenario', 'geometry']
amn = amn[amn_cols]

In [ ]:
amn = gpd.GeoDataFrame(data=amn.drop(columns=['geometry']), geometry=amn['geometry'])

In [ ]:
amn['id'] = amn['id'].astype(str)

In [ ]:
amn.loc[amn['scenario']=='actual', ['id', 'Category', 'geometry']].to_parquet('/app/assets/amenities/actual.parquet')
amn.loc[:, ['id', 'Category', 'geometry']].to_parquet('/app/assets/amenities/future.parquet')

In [ ]:
amn.to_parquet('/app/assets/amenities/amenities.parquet')